# Spark ML serving with MLeap in Azure Machine Learning

In this notebook, we'll host Spark ML serving (online prediction) by using MLeap.

Spark ML pipeline model (which is trained on Apache Spark) can be converted into MLeap format (MLeap bundle). Using MLeap format, you can then serve Spark MLlib model on a lightweight single container without Apache Spark installation.

> Note : I note that you cannot always take this method, because not all model in Spark can be serialized into MLeap. (such as, LightGBM model built with SynapseML library)

To run this notebook,

1. Create new "Machine Learning" resource in [Azure Portal](https://portal.azure.com/).
2. Install Azure Machine Learning CLI v2 on Ubuntu as follows

```
# install Azure CLI
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
# install AML CLI extension
az extension add --name ml
```

> Note : See [here](https://tsmatz.wordpress.com/2019/03/04/spark-ml-pipeline-serving-inference-by-azure-machine-learning-service/) for other deployment options for Spark ML model.

## Connect to Azure Machine Learning workspace

Login to Azure and prepare for connecting to Azure Machine Learning (AML) workspace.<br>
Please fill the following subscription id, AML workspace name, and resource group name.

In [ ]:
!az login

In [ ]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [ ]:
my_resource_group = "{AML_RESOURCE_GROUP_NAME}"
my_workspace = "{AML_WORSPACE_NAME}"

## Train model and convert to MLeap format

First train Spark ML pipeline on Apache Spark, and convert model into MLeap bundle (.zip file).

In this example, we use MLeap pipeline bundle, ```flight-delay-classify.zip```, which is trained in [this Databricks exercise](https://tsmatz.github.io/azure-databricks-exercise/exercise05-mleap.html).<br>
This pipeline uses inputs for flight and weather information (such as, aircraft carrier, depature/arrival time, depature/arrival wind speed, depature/arrival visibility, etc) and then predicts flight arrival's delay over 15 minutes by 0 or 1.

Now we register this trained MLeap bundle into Azure Machine Learning.

In [2]:
!az ml model create --name fight_delay_mleap_model \
  --version 1 \
  --path ./flight-delay-classify.zip \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Uploading flight-delay-classify.zip (< 1 MB): 100%|█| 165k/165k [00:00<00:00, 2.


{
  "creation_context": {
    "created_at": "2022-08-29T00:44:48.804476+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-08-29T00:44:48.804476+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/models/fight_delay_mleap_model/versions/1",
  "name": "fight_delay_mleap_model",
  "path": "azureml://subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/workspaces/ws01/datastores/workspaceblobstore/paths/LocalUpload/7c2e0fe6bee0b71f677559eeb284167c/flight-delay-classify.zip",
  "properties": {},
  "resourceGroup": "AML-rg",
  "tags": {},
  "type": "custom_model",
  "version": "1"
}


## Create a managed endpoint

Now create a managed endpoint. **Fill the following endpoint name**, which must be unique in DNS.

Afterwards, we will deploy inferencing container image on this endpoint.

In [3]:
%%writefile managed_endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: {FILL_UNIQUE_ENDPOINT_NAME}
auth_mode: key

Writing managed_endpoint.yml


In [4]:
!az ml online-endpoint create --file managed_endpoint.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

{
  "auth_mode": "key",
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/onlineEndpoints/aml-mleap-test01",
  "identity": {
    "principal_id": "db05a1d4-d34c-44f9-b77e-ccc9e09d22fd",
    "tenant_id": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "type": "system_assigned"
  },
  "kind": "Managed",
  "location": "eastus",
  "mirror_traffic": {},
  "name": "aml-mleap-test01",
  "properties": {
    "AzureAsyncOperationUri": "https://management.azure.com/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:b08a1272-2af0-4e5c-bcdf-bb474ba3c272:7f419933-3dc5-4ca2-b76f-dc9b7b1666d1?api-version=2022-02-01-preview",
    "azureml.onlineendpointid": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/aml-rg/providers/microsoft.machinelearningservices/workspaces/ws01/onlineendpoints/aml-mleap-test01"
  },
  "pr

## Deploy inferencing with MLeap runtime

Now we deploy inferencing image on this endpoint.

In this example, we use the following pre-built MLeap serving container image with Spring Boot framework.

MLeap Serving (Spring Boot) :<br>
[https://combust.github.io/mleap-docs/mleap-serving/](https://combust.github.io/mleap-docs/mleap-serving/)

In the following settings :
- The model (which is registered above) is mounted as ```/models```. This directory is used in model loading later.
- The port number 65327 is default port for MLeap Spring Boot Serving. (See [here](https://combust.github.io/mleap-docs/mleap-serving/).)<br>
  The url ```http://localhost:65327/actuator``` is then used for liveness endpoint.
- The endpoint ```http://localhost:65327/``` is exposed by ```https://{ENDPOINT ADDRESS}/``` and ```https://{ENDPOINT ADDRESS}/models``` is returned as scoring url by the following ```environment/inference_config/scoring_route``` setting.

In [5]:
%%writefile managed_deployment.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: flight-delay-deployment
endpoint_name: {FILL_UNIQUE_ENDPOINT_NAME}
model: azureml:fight_delay_mleap_model@latest
model_mount_path: /models
environment:
  image: docker.io/combustml/mleap-spring-boot:0.21.0-SNAPSHOT
  inference_config:
    liveness_route:
      port: 65327
      path: /actuator
    readiness_route:
      port: 65327
      path: /actuator
    scoring_route:
      port: 65327
      path: /models
instance_type: Standard_DS2_v2
instance_count: 1

Writing managed_deployment.yml


In [6]:
!az ml online-deployment create --file managed_deployment.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --all-traffic

All traffic will be set to deployment flight-delay-deployment once it has been provisioned.
If you interrupt this command or it times out while waiting for the provisioning, you can try to set all the traffic to this deployment later once its has been provisioned.
Check: endpoint aml-mleap-test01 exists
Creating/updating online deployment flight-delay-deployment .......................................................................Done (6m 17s)
{
  "app_insights_enabled": false,
  "endpoint_name": "aml-mleap-test01",
  "environment": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/environments/CliV2AnonymousEnvironment/versions/4fb6ecfd15e1fc56568a1dd7ba90693e",
  "environment_variables": {},
  "instance_count": 1,
  "instance_type": "Standard_DS2_v2",
  "model": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/wor

## Get endpoint url and key

Get endpoint url by the following command. (**Fill the following endpoint name**)

In [7]:
endpoint_name = "{FILL_UNIQUE_ENDPOINT_NAME}"

In [8]:
!az ml online-endpoint show \
  --name $endpoint_name \
  --query scoring_uri \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

"https://aml-mleap-test01.eastus.inference.ml.azure.com/models"


Get authorization key for this endpoint by the following command.

In [9]:
!az ml online-endpoint get-credentials \
  --name $endpoint_name \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

{
  "primaryKey": "0xtXv4OaKnLHBuAqeqGA0vNjs0GPa4Ai",
  "secondaryKey": "mqszbFnGi9yrzIzuwbpXwhS8EgJbdR2I"
}


## Test web service

Now test your web service.

First, we must load model in ```models/flight-delay-classify.zip``` with the name of ```flight-delay``` as follows.

> Note : You can check metadata (such as, schemas in leap frame, etc) of the loaded model by ```GET http://{ENDPOINT_ADDRESS}/models/flight-delay/meta```

In [10]:
endpoint_url = "{FILL_ENDPOINT_URL}"
# Example : endpoint_url = "https://aml-mleap-test01.eastus.inference.ml.azure.com/models"
authorization_key = "{FILL_AUTHORIZATION_KEY}"
# Example : authorization_key = "0xtXv4OaKnLHBuAqeqGA0vNjs0GPa4Ai"

In [11]:
import requests
import json

# Invoke web service !
headers = {
    'Content-Type':'application/json',
    'Authorization':('Bearer '+ authorization_key)
}
body = "{\"modelName\":\"flight-delay\",\"uri\":\"file:/models/flight-delay-classify.zip\",\"config\":{\"memoryTimeout\":900000,\"diskTimeout\":900000},\"force\":false}"
http_res = requests.post(
    endpoint_url,
    body,
    headers = headers)

# Show results
print("Status code : {}".format(http_res.status_code))
result_body = json.loads(http_res.text)
print(result_body)

Status code : 202
{'name': 'flight-delay', 'uri': 'file:/models/flight-delay-classify.zip', 'config': {'memoryTimeout': '900000', 'diskTimeout': '900000'}}


Predict flight delay with this endpoint as follows. (Call online inferencing.)

In the following example, the predicted result is ```0.0``` (which means "not delayed").

In [12]:
import requests
import json

# Read leap frame
with open('leap-frame.txt') as f:
    body = f.read()

# Invoke web service !
headers = {
    'Content-Type':'application/json',
    'Authorization':('Bearer '+ authorization_key)
}
http_res = requests.post(
    endpoint_url + "/flight-delay/transform",
    body,
    headers = headers)

# Show results
# Show results
print("Status code : {}".format(http_res.status_code))
result_body = json.loads(http_res.text)
print(result_body)

Status code : 200
{'schema': {'fields': [{'name': 'AltimeterOrigin', 'type': 'double'}, {'name': 'RelativeHumidityOrigin', 'type': 'double'}, {'name': 'DryBulbCelsiusDest', 'type': 'double'}, {'name': 'AltimeterDest', 'type': 'double'}, {'name': 'UNIQUE_CARRIER', 'type': 'string'}, {'name': 'ORIGIN', 'type': 'string'}, {'name': 'DewPointCelsiusDest', 'type': 'double'}, {'name': 'DEST', 'type': 'string'}, {'name': 'DAY_OF_WEEK', 'type': 'double'}, {'name': 'MONTH', 'type': 'double'}, {'name': 'RelativeHumidityDest', 'type': 'double'}, {'name': 'CRS_DEP_TIME', 'type': 'double'}, {'name': 'VisibilityDest', 'type': 'double'}, {'name': 'ARR_DEL15', 'type': 'string'}, {'name': 'VisibilityOrigin', 'type': 'double'}, {'name': 'WindSpeedOrigin', 'type': 'double'}, {'name': 'DewPointCelsiusOrigin', 'type': 'double'}, {'name': 'DryBulbCelsiusOrigin', 'type': 'double'}, {'name': 'WindSpeedDest', 'type': 'double'}, {'name': 'CRS_ARR_TIME', 'type': 'double'}, {'name': 'Indexed_UNIQUE_CARRIER', 'type

## Clean up

Remove endpoint.

In [ ]:
!az ml online-endpoint delete \
  --name $endpoint_name \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --yes